# Multiply-connected ports

> Sax supports multiply-connected ports, but be aware that those introduce gain.

## Imports

In [ ]:
import jax.numpy as jnp
import matplotlib.pyplot as plt

import sax

## MZI with multi-link nets

Normally an MZI requires explicit coupler components to split and combine signals.
With **multi-link nets**, a single port can connect to multiple other ports.
The KLU backend handles this natively in its sparse matrix formulation.

```svgbob
               .---[top]---.
              /              \
  [input] --+                +-- [output]
              \              /
               '---[btm]---'
```

Here `input,out0` connects to both `top,in0` and `btm,in0` (a 1-to-2 split),
and both `top,out0` and `btm,out0` connect to `output,in0` (a 2-to-1 combine).
We express this with `nets` — a list of point-to-point connections where the same
port may appear more than once:

In [ ]:
netlist = {
    "instances": {
        "input": "waveguide",
        "top": "waveguide",
        "btm": "waveguide",
        "output": "waveguide",
    },
    "nets": [
        {"p1": "input,out0", "p2": "top,in0"},
        {"p1": "input,out0", "p2": "btm,in0"},  # multi-link: same source port
        {"p1": "top,out0", "p2": "output,in0"},
        {"p1": "btm,out0", "p2": "output,in0"},  # multi-link: same target port
    ],
    "ports": {
        "in0": "input,in0",
        "out0": "output,out0",
    },
}

Multi-link nets require the `klu` backend:

In [ ]:
mzi, _ = sax.circuit(
    netlist,
    models={"waveguide": sax.models.straight},
    backend="klu",
)

mzi()

## Wavelength sweep

With different arm lengths we get the classic MZI interference pattern:

In [ ]:
wl = jnp.linspace(1.51, 1.59, 1000)

S = mzi(wl=wl, top={"length": 25.0}, btm={"length": 15.0})

plt.plot(wl * 1e3, jnp.abs(S["in0", "out0"]) ** 2)
plt.xlabel("λ [nm]")
plt.ylabel("T")
plt.title("MZI transmission (multi-link nets)")
plt.show()

## Why?
Using multiply connected ports is NOT recommended. The only reason why we support this is that sometimes extracted netlists (e.g. from GDSFactory) contain layout errors where multiple ports are connected together due to a faulty placement. However, if you've determined that this won't have an influence on the simulation SAX won't stop you simulating the circuit.

The example above is maybe not the best as there we're mis-using the feature. In general ONLY simulate netlists with multiply-connected components if one of the arms of the multiply connected ports leads to nowhere you're interested in.